#  PHASE 1: PROJECT 


##  Project Title: Aircraft Safety Intelligence: A Data-Driven Risk Analysis for Strategic Fleet Investment

##  Goals
#### 1. Evaluate Aircraft Risk Profiles by Type, Make, and Model  
Use historical accident and incident data (`Make`, `Model`, `Aircraft.Category`, `Engine.Type`) to identify which 
aircraft types have the lowest rates of:
- `Fatal injuries`  
- `Substantial damage`  
- `Serious incidents`  

#### 2. Analyze Factors Contributing to Aircraft Accidents  
Explore key causes and conditions contributing to aircraft accidents by examining:

- `Weather.Condition` (e.g., VMC vs IMC)  
- `Broad.phase.of.flight` (e.g., Takeoff, Landing)  
- `Injury.Severity`, `Total.Fatal.Injuries`, `Total.Serious.Injuries`  
- `Purpose.of.flight`

#### 3. Assess Geographical and Operational Risk  
Identify high-risk locations using `Location` and `Country`, and compare safety across different flight purposes (`Purpose.of.flight`, `Airport.Name`) to highlight safer:

- `Operational zones`  
- `Use cases (e.g., commercial, personal, ferry, instructional)`

#### 4. Provide Strategic Recommendations for Safer Aircraft Investment  
Synthesize findings into **three actionable recommendations** to guide the aviation division’s initial investment strategy by prioritizing:
- `Aircraft with lower incident frequency`  
- `Models with minimal injury severity`  
- `Safer categories based on empirical evidence`  


## Data

### 1.1: Data Overview

The dataset used in this analysis is sourced from the **National Transportation Safety Board (NTSB) Aviation Accident Database**. It contains detailed records from **1962 onwards** involving:

- Civil aviation **accidents** and selected **incidents**
- Events that occurred within the **United States**, its **territories**, **international waters** and **other-countries**

This dataset includes variables related to aircraft type, damage severity, injury outcomes, weather conditions, flight phases, geographic locations, and more enabling robust analysis of aviation safety trends over time.


### 1.2: Data Description

##### 1.2.1: Importing the dataset

In [1]:
#Importing the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#Reading the dataset and checking top five rows
df = pd.read_csv('../data/AviationData.csv', encoding='latin1', low_memory=False)
df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


**The file isn't encoded in UTF-8, As a result, trying to load it normally causes an error. The use of encoding="latin1" solved the problem.**

##### 1.2.2: Basic structure

In [3]:
df.shape

(88889, 31)

In [4]:
df.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status',
       'Publication.Date'],
      dtype='object')

##### 1.2.3: Overview of column types and non-null values

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52790 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87572 non-null  object 
 14  Make                    88826 non-null

##### 1.2.4 Summary statistics numerical

In [6]:
df.describe(include='number').T

,count,mean,std,min,25%,50%,75%,max
Number.of.Engines,82805.0,1.146585,0.446510,0.0,1.0,1.0,1.0,8.0
Total.Fatal.Injuries,77488.0,0.647855,5.485960,0.0,0.0,0.0,0.0,349.0
Total.Serious.Injuries,76379.0,0.279881,1.544084,0.0,0.0,0.0,0.0,161.0
Total.Minor.Injuries,76956.0,0.357061,2.235625,0.0,0.0,0.0,0.0,380.0
Total.Uninjured,82977.0,5.325440,27.913634,0.0,0.0,1.0,2.0,699.0


##### 1.2.5 Summary statistics categorical

In [7]:
df.describe(include='O')

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Amateur.Built,Engine.Type,FAR.Description,Schedule,Purpose.of.flight,Air.carrier,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
count,88889,88889,88889,88889,88837,88663,34382,34373,50249,52790,...,88787,81812,32023,12582,82697,16648,84397,61724,82508,75118
unique,87951,2,88863,14782,27758,219,25589,27154,10375,24871,...,2,13,31,3,26,13590,4,12,17075,2924
top,20001212X19172,Accident,WPR23LA041,1982-05-16,"ANCHORAGE, AK",United States,332739N,0112457W,NONE,Private,...,No,Reciprocating,091,NSCH,Personal,Pilot,VMC,Landing,Probable Cause,25-09-2020
freq,3,85015,2,25,434,82248,19,24,1488,240,...,80312,69530,18221,4474,49448,258,77303,15428,61754,17019


##### 1.2.6 Missing values

In [8]:
#missing values as sum
df.isnull().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     52
Country                     226
Latitude                  54507
Longitude                 54516
Airport.Code              38640
Airport.Name              36099
Injury.Severity            1000
Aircraft.damage            3194
Aircraft.Category         56602
Registration.Number        1317
Make                         63
Model                        92
Amateur.Built               102
Number.of.Engines          6084
Engine.Type                7077
FAR.Description           56866
Schedule                  76307
Purpose.of.flight          6192
Air.carrier               72241
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          4492
Broad.phase.of.flight     27165
Report.Status              6381
Publication.Date          13771
dtype: i

In [9]:
#missing values as mean
df.isnull().mean()*100

Event.Id                   0.000000
Investigation.Type         0.000000
Accident.Number            0.000000
Event.Date                 0.000000
Location                   0.058500
Country                    0.254250
Latitude                  61.320298
Longitude                 61.330423
Airport.Code              43.469946
Airport.Name              40.611324
Injury.Severity            1.124999
Aircraft.damage            3.593246
Aircraft.Category         63.677170
Registration.Number        1.481623
Make                       0.070875
Model                      0.103500
Amateur.Built              0.114750
Number.of.Engines          6.844491
Engine.Type                7.961615
FAR.Description           63.974170
Schedule                  85.845268
Purpose.of.flight          6.965991
Air.carrier               81.271023
Total.Fatal.Injuries      12.826109
Total.Serious.Injuries    14.073732
Total.Minor.Injuries      13.424608
Total.Uninjured            6.650992
Weather.Condition          5

### 1.3: Data Summary
- The dataset consists of 88,889 records and 31 columns, capturing civil aviation accidents and incidents from 1962 to 2022. The dataset includes attributes such as event date, location, aircraft specifications [`make`, `model`, `engine type`], flight purpose, injury counts, and weather conditions.
- Numerical summaries indicate that while most flights involved one engine, the average number of fatal injuries per event is relatively low (mean = 0.65), though some incidents resulted in high casualty counts (up to 349 fatalities). However, the dataset contains significant missing values in key fields like `Latitude`, `Longitude`, `Aircraft.Category`, `Engine.Type`, `FAR.Description`, and `Schedule`, with some missing rates exceeding 60%. These gaps will be considered during data cleaning and  data analysis to ensure accurate and meaningful insights are drawn from the data without distorting meaning.


### 1.4: Data Cleaning

##### 1.4.1 Removing columns which are not useful to our analysis and objectives
To streamline the dataset and align with project goals, several columns can be dropped due to high missing values, low relevance, or redundancy. `Event.Id`, `Accident.Numbe`r, and `Registration.Number` are unique identifiers that don't support aggregation. `Latitude` and `Longitude` have over **61%** missing data and inconsistent formats, with **Location** and **Country** providing better geographic context. `Airport.Code`, `FAR.Description`, `Schedule`, and `Air.carrier` can be removed due to low impact on the analysis and objectives. On the other hand `Report.Status` and `Publication.Date` will be excluded as they are administrative and not essential for evaluating aircraft risk, accident causes, or strategic recommendations. This refinement ensures the dataset is cleaner and more focused on the objectives.

In [10]:
#creating a list and dropping the columns
print(f"Shape Before: {df.shape}")
columns_to_drop = [ 'Event.Id',
                   'Accident.Number', 
                   'Latitude', 
                   'Longitude',
                   'Airport.Code', 
                   'Registration.Number',
                   'FAR.Description', 
                   'Schedule', 
                   'Air.carrier', 
                   'Report.Status', 
                   'Publication.Date'
]
df.drop(columns=columns_to_drop, inplace=True)
print(f"Shape After: {df.shape}")

Shape Before: (88889, 31)
Shape After: (88889, 20)


**We have removed 11 columns from the dataset, reducing the total number of columns from 31 to 20, while retaining all 88,889 rows**

In [11]:
for col in df.columns:
  print(f"Unique values in column '{col}':")
  print(df[col].unique())
  print("\n********")

Unique values in column 'Investigation.Type':
['Accident' 'Incident']

********
Unique values in column 'Event.Date':
['1948-10-24' '1962-07-19' '1974-08-30' ... '2022-12-22' '2022-12-26'
 '2022-12-29']

********
Unique values in column 'Location':
['MOOSE CREEK, ID' 'BRIDGEPORT, CA' 'Saltville, VA' ... 'San Manual, AZ'
 'Auburn Hills, MI' 'Brasnorte, ']

********
Unique values in column 'Country':
['United States' nan 'GULF OF MEXICO' 'Puerto Rico' 'ATLANTIC OCEAN'
 'HIGH ISLAND' 'Bahamas' 'MISSING' 'Pakistan' 'Angola' 'Germany'
 'Korea, Republic Of' 'Martinique' 'American Samoa' 'PACIFIC OCEAN'
 'Canada' 'Bolivia' 'Mexico' 'Dominica' 'Netherlands Antilles' 'Iceland'
 'Greece' 'Guam' 'Australia' 'CARIBBEAN SEA' 'West Indies' 'Japan'
 'Philippines' 'Venezuela' 'Bermuda' 'San Juan Islands' 'Colombia'
 'El Salvador' 'United Kingdom' 'British Virgin Islands' 'Netherlands'
 'Costa Rica' 'Mozambique' 'Jamaica' 'Panama' 'Guyana' 'Norway'
 'Hong Kong' 'Portugal' 'Malaysia' 'Turks And Caicos I

##### 1.4.2 Convert date column to datetime type 

In [12]:
#checking the current type
df['Event.Date'].dtype

dtype('O')

In [13]:
#Coverting the event.date into a datetime type
df['Event.Date'] = pd.to_datetime(df['Event.Date'], errors='coerce')
df['Event.Date'].dtype

dtype('<M8[ns]')

In [14]:
#checking for null
df['Event.Date'].isna().sum()

0

##### 1.4.3 Cleaning Categorical Columns 
 **Here we will impute/drop null values/rows and standardize unique values.**

###### 1.4.3.1: Column-->Location

In [15]:
#checking proportion of null values
df['Location'].isna().mean()*100

0.05849992687509141

In [16]:
df.dropna(subset=['Location'], inplace=True)

**The `Location` column had 52 missing values, representing just 0.06% of the dataset.
Given the minimal proportion, dropping these rows is unlikely to affect the analysis significantly.**





###### 1.4.3.2: Column-->Country

In [17]:
#checking proportion of null values
df['Country'].isna().mean()*100

0.25327284802503464

In [18]:
#checking unique values
df['Country'].unique()

array(['United States', nan, 'GULF OF MEXICO', 'Puerto Rico',
       'ATLANTIC OCEAN', 'HIGH ISLAND', 'Bahamas', 'MISSING', 'Angola',
       'Germany', 'Korea, Republic Of', 'American Samoa', 'PACIFIC OCEAN',
       'Canada', 'Bolivia', 'Mexico', 'Dominica', 'Netherlands Antilles',
       'Iceland', 'Greece', 'Guam', 'Australia', 'CARIBBEAN SEA',
       'West Indies', 'Japan', 'Philippines', 'Venezuela', 'Bermuda',
       'San Juan Islands', 'Colombia', 'El Salvador', 'United Kingdom',
       'British Virgin Islands', 'Netherlands', 'Costa Rica',
       'Mozambique', 'Jamaica', 'Panama', 'Guyana', 'Norway', 'Portugal',
       'Malaysia', 'Turks And Caicos Islands', 'Northern Mariana Islands',
       'Dominican Republic', 'Suriname', 'Honduras', 'Congo', 'Belize',
       'Guatemala', 'France', 'St Vincent And The Grenadines', 'Haiti',
       'Papua New Guinea', 'Cayman Islands', 'Sweden', 'Taiwan',
       'Senegal', 'Barbados', 'BLOCK 651A', 'Brazil', 'Mauritius',
       'Argentina', 'K

In [19]:
#going through the list above, the names below are not indicative of any country in particular
invalid_names = [ 'GULF OF MEXICO', 'ATLANTIC OCEAN', 'HIGH ISLAND', 'PACIFIC OCEAN',
    'CARIBBEAN SEA', 'San Juan Islands', 'BLOCK 651A',
    'Wolseley', 'Obyan', 'AY', 'Cayenne', 'UN']

df['Country'] = df['Country'].replace(invalid_names, 'Unknown')

country_name_var = {
    'Ivory Coast': "Côte d'Ivoire",
    "Cote D'ivoire": "Côte d'Ivoire",
    'South Korean': 'South Korea',
    'Korea, Republic Of': 'South Korea',
    'Central Africa': 'Central Africa Republic',
    'Bosnia And Herzegovina': 'Bosnia and Herzegovina',
    'St Vincent And The Grenadines': 'Saint Vincent and the Grenadines',
    'Antigua And Barbuda': 'Antigua and Barbuda',
    'Turks And Caicos': 'Turks and Caicos Islands',
    'Turks And Caicos Islands': 'Turks and Caicos Islands',
    'Great Britain': 'United Kingdom',
    'Scotland': 'United Kingdom',
}

df['Country'] = df['Country'].replace(country_name_var)
df['Country'] = df['Country'].fillna('Unknown')

**For the column `Country`, I started by replacing 12 invalid geographic entries with 'Unknown', 
and unified inconsistent country names like 'Ivory Coast' → 'Côte d'Ivoire' and finally filled the missing values with 'Unknown'. Since 'Unknown' is already a valid category in the column,dropping the null values was not necessary and could have led to data loss.**

###### 1.4.3.3: Column-->Airport.Name

In [20]:
#checking proportion of null values
df['Airport.Name'].isna().mean()*100

40.578812882019875

In [21]:
#checking unique values
df['Airport.Name'].unique()

array([nan, 'BLACKBURN AG STRIP', 'HANOVER', ...,
       'HAWKINSVILLE-PULASKI COUNTY', 'Lewiston Municipal Airport',
       'WICHITA DWIGHT D EISENHOWER NT'], dtype=object)

In [22]:
#Imputing the nan values with UNKNOWN
df['Airport.Name'].fillna('UNKNOWN', inplace=True)
df['Airport.Name'].isna().mean()*100

0.0

###### 1.4.3.4: Column-->Injury.Severity

In [23]:
#checking proportion of null values
df['Injury.Severity'].isna().mean()*100

1.1256571023334871

In [24]:
#checking unique values
df['Injury.Severity'].unique()

array(['Fatal(2)', 'Fatal(4)', 'Fatal(3)', 'Fatal(1)', 'Non-Fatal',
       'Incident', 'Fatal(8)', 'Fatal(78)', 'Fatal(7)', 'Fatal(6)',
       'Fatal(5)', 'Fatal(153)', 'Fatal(12)', 'Fatal(14)', 'Fatal(23)',
       'Fatal(10)', 'Fatal(11)', 'Fatal(9)', 'Fatal(17)', 'Fatal(13)',
       'Fatal(29)', 'Fatal(70)', 'Unavailable', 'Fatal(135)', 'Fatal(31)',
       'Fatal(256)', 'Fatal(25)', 'Fatal(82)', 'Fatal(156)', 'Fatal(28)',
       'Fatal(18)', 'Fatal(43)', 'Fatal(15)', 'Fatal(270)', 'Fatal(144)',
       'Fatal(174)', 'Fatal(111)', 'Fatal(131)', 'Fatal(20)', 'Fatal(73)',
       'Fatal(27)', 'Fatal(34)', 'Fatal(87)', 'Fatal(30)', 'Fatal(16)',
       'Fatal(47)', 'Fatal(56)', 'Fatal(37)', 'Fatal(132)', 'Fatal(68)',
       'Fatal(54)', 'Fatal(52)', 'Fatal(65)', 'Fatal(72)', 'Fatal(160)',
       'Fatal(189)', 'Fatal(123)', 'Fatal(33)', 'Fatal(110)',
       'Fatal(230)', 'Fatal(97)', 'Fatal(349)', 'Fatal(125)', 'Fatal(35)',
       'Fatal(228)', 'Fatal(75)', 'Fatal(104)', 'Fatal(229)', 'Fatal

In [25]:
df.dropna(subset=['Injury.Severity'], inplace=True)

**The `Injury.Severity` column had null rows representing 1.1% of the dataset.
Given the minimal proportion, dropping these rows is unlikely to affect the analysis significantly.**

###### 1.4.3.5: Column-->Aircraft.damage

In [26]:
#checking proportion of null values
df['Aircraft.damage'].isna().mean()*100

3.0533829707298747

In [27]:
#checking unique values
df['Aircraft.damage'].unique()

array(['Destroyed', 'Substantial', 'Minor', nan, 'Unknown'], dtype=object)

In [28]:
df['Aircraft.damage'].value_counts()

Substantial    63951
Destroyed      18537
Minor           2575
Unknown           92
Name: Aircraft.damage, dtype: int64

In [29]:
#Imputing using the string Unknown
df['Aircraft.damage'] = df['Aircraft.damage'].fillna('Unknown')

**I decided to impute with the string "Unknown" as opposed to "Substantial" which is the mode since it will help preserve the integrity of actual damage distribution and avoid biasing results.**


###### 1.4.3.6: Column-->Aircraft.Category

In [30]:
#checking proportion of null values
df['Aircraft.Category'].isna().mean()*100

64.2587975454535

In [31]:
#checking unique values
df['Aircraft.Category'].unique()

array([nan, 'Airplane', 'Helicopter', 'Glider', 'Balloon', 'Gyrocraft',
       'Ultralight', 'Unknown', 'Blimp', 'Powered-Lift', 'Weight-Shift',
       'Powered Parachute', 'Rocket', 'WSFT', 'ULTR'], dtype=object)

In [32]:
df['Aircraft.Category'].value_counts()

Airplane             26796
Helicopter            3373
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 11
WSFT                     9
Powered-Lift             5
Blimp                    4
Rocket                   1
ULTR                     1
Name: Aircraft.Category, dtype: int64

In [33]:
#Imputing null with Unknown string
df['Aircraft.Category'].fillna("Unknown", inplace=True)

**I decided to impute with the string "Unknown" as opposed to mode of the column:
This will help preserve the integrity of the distribution and avoid biasing results.**

###### 1.4.3.7: Column-->Make, Model, Amateur.Built

In [34]:
#Creating a list of columns describing aircraft Build Characteristics 
build_columns = ['Make', 'Model', 'Amateur.Built']

#checking null proportion
for col in build_columns:
        print(f"Column Name: {col}")
        print(df[col].isna().mean()*100)
        print("*****")

Column Name: Make
0.05236973029588898
*****
Column Name: Model
0.08197001263704362
*****
Column Name: Amateur.Built
0.11157029497819826
*****


In [36]:
#dropping the null rows
df.dropna(subset=['Make', 'Model', 'Amateur.Built'], inplace=True)

In [37]:
#checking unique items value counts
for col in build_columns:
        print(f"Column Name: {col}")
        print(df[col].value_counts())
        print("*****")

Column Name: Make
Cessna                      22193
Piper                       12012
CESSNA                       4838
Beech                        4321
PIPER                        2813
                            ...  
Worcester Everett               1
CIRRUS DESIGN                   1
TWENTY SECOND STREET LLC        1
Scheicher                       1
Aero Tek, Inc.                  1
Name: Make, Length: 8197, dtype: int64
*****
Column Name: Model
152                  2360
172                  1741
172N                 1161
PA-28-140             930
150                   825
                     ... 
RS15                    1
HU16A                   1
SUPER PETREL LS         1
551 (Citation II)       1
LAKE MODEL 250          1
Name: Model, Length: 12165, dtype: int64
*****
Column Name: Amateur.Built
No     79215
Yes     8436
Name: Amateur.Built, dtype: int64
*****


In [45]:
# Stripping whitespace and converting to lower case for Make, upper case for Model if needed
df['Make'] = df['Make'].str.strip().str.lower()
df['Model'] = df['Model'].str.strip().str.upper()

#standardizing names for make
make_map = {
    'cessna aircraft co': 'cessna',
    'cessna aircraft company': 'cessna',
    'piper aircraft inc': 'piper',
    'piper aircraft corporation': 'piper',
    'robinson helicopter company': 'robinson',
    'robinson helicopter co': 'robinson',
    'md helicopters inc': 'md helicopter',
}
df['Make'] = df['Make'].replace(make_map)



In [39]:
#checking unique items value counts
for col in build_columns:
        print(f"Column Name: {col}")
        print(df[col].value_counts())
        print("*****")

Column Name: Make
cessna                 27066
piper                  14862
beech                   5345
bell                    2696
boeing                  2239
                       ...  
young robert herman        1
elan glaser dirks          1
john a brewer              1
meadows                    1
turnbloom                  1
Name: Make, Length: 7544, dtype: int64
*****
Column Name: Model
152               2360
172               1741
172N              1161
PA-28-140          930
150                825
                  ... 
172-B                1
CC-18-180            1
MD369                1
MU-2B-35J            1
LAKE MODEL 250       1
Name: Model, Length: 11497, dtype: int64
*****
Column Name: Amateur.Built
No     79215
Yes     8436
Name: Amateur.Built, dtype: int64
*****


**To clean and standardize the aircraft build attributes, I first focused on the key columns: `Make`, `Model`, and `Amateur.Built`. Assessing the proportion of missing values in each column, I realized they are below 0.12%, and determined it was reasonable to drop these rows without significantly impacting the dataset. After dropping the nulls, I examined the distribution of unique values using `value_counts()` for each column. To ensure consistency, I standardized the text formatting by stripping whitespace and converting all entries in the `Make` column to lowercase and those in the `Model` column to uppercase. Finally, I normalized manufacturer names by replacing known variations with standardized names for example, mapping `'cessna aircraft company'` and `'cessna aircraft co'` to `'cessna'`, and similar adjustments for Piper, Robinson, and MD Helicopters thus improving consistency for future grouping and analysis.**


###### 1.4.3.8: Column-->Engine.Type

In [47]:
#checking proportion of null values
df['Engine.Type'].isna().mean()*100

7.006195023445254

In [48]:
#checking unique values
df['Engine.Type'].unique()

array(['Reciprocating', nan, 'Turbo Fan', 'Turbo Shaft', 'Unknown',
       'Turbo Prop', 'Turbo Jet', 'None', 'Electric', 'Hybrid Rocket',
       'Geared Turbofan', 'LR', 'NONE', 'UNK'], dtype=object)

In [49]:
df['Engine.Type'].value_counts()

Reciprocating      69446
Turbo Shaft         3597
Turbo Prop          3370
Turbo Fan           2348
Unknown             2017
Turbo Jet            698
None                  19
Electric               8
NONE                   2
LR                     2
UNK                    1
Geared Turbofan        1
Hybrid Rocket          1
Name: Engine.Type, dtype: int64

In [50]:
#Imputing Null with UNK and standardizing None to NONE and also Unknown to UNK
df['Engine.Type'].fillna('UNK', inplace=True)
df['Engine.Type'] = df['Engine.Type'].replace({'Unknown': 'UNK', 'None': 'NONE'})

**The `Engine.Type` column contains inconsistent representations of missing or unknown values, such as `'nan'`, `'Unknown'`, and `'UNK'`, which all indicate a lack of engine type. Additionally, for aircraft with no engine, the values `'NONE'` and `'None'` are used interchangeably. To ensure consistency, we standardize these entries by replacing `'nan'` with `'UNK'`, and unifying all missing or unknown types under the `'UNK'` label. Similarly, we consolidate the `'NONE'` and `'None'` values into a single representation.**

###### 1.4.3.9: Column-->Purpose.of.flight

In [51]:
#checking proportion of null values
df['Purpose.of.flight'].isna().mean()*100

5.861884062931398

In [52]:
#checking unique values
df['Purpose.of.flight'].unique()

array(['Personal', nan, 'Business', 'Instructional', 'Unknown', 'Ferry',
       'Executive/corporate', 'Aerial Observation', 'Aerial Application',
       'Public Aircraft', 'Skydiving', 'Other Work Use', 'Positioning',
       'Flight Test', 'Air Race/show', 'Air Drop',
       'Public Aircraft - Federal', 'Glider Tow',
       'Public Aircraft - Local', 'External Load',
       'Public Aircraft - State', 'Banner Tow', 'Firefighting',
       'Air Race show', 'PUBS', 'ASHO', 'PUBL'], dtype=object)

In [53]:
df['Purpose.of.flight'].value_counts()

Personal                     49372
Instructional                10586
Unknown                       6744
Aerial Application            4709
Business                      4009
Positioning                   1643
Other Work Use                1264
Ferry                          808
Aerial Observation             791
Public Aircraft                719
Executive/corporate            550
Flight Test                    398
Skydiving                      182
External Load                  123
Public Aircraft - Federal      104
Banner Tow                     101
Air Race show                   99
Public Aircraft - Local         74
Public Aircraft - State         63
Air Race/show                   59
Glider Tow                      53
Firefighting                    40
Air Drop                        11
ASHO                             6
PUBS                             4
PUBL                             1
Name: Purpose.of.flight, dtype: int64

In [54]:
#The null values are not many, can proceed to impute the string Unknown since it already exists
df['Purpose.of.flight'].fillna('Unknown', inplace=True)
df['Purpose.of.flight'] = df['Purpose.of.flight'].replace({
    'Air Race show': 'ASHO',
    'Air Race/show': 'ASHO',
    'Public Aircraft - Federal': 'Public Aircraft',
    'Public Aircraft - Local': 'Public Aircraft',
    'Public Aircraft - State': 'Public Aircraft',
    'PUBS': 'Public Aircraft',
    'PUBL': 'Public Aircraft'
})
df['Purpose.of.flight'].unique()

array(['Personal', 'Unknown', 'Business', 'Instructional', 'Ferry',
       'Executive/corporate', 'Aerial Observation', 'Aerial Application',
       'Public Aircraft', 'Skydiving', 'Other Work Use', 'Positioning',
       'Flight Test', 'ASHO', 'Air Drop', 'Glider Tow', 'External Load',
       'Banner Tow', 'Firefighting'], dtype=object)

**The `Purpose.of.flight` column contained a small proportion of missing values, which made it reasonable to impute them using the string `'Unknown'` — a label already present in the dataset. After imputation, further cleaning was performed to standardize similar or redundant category labels. For example, different variants of air show-related purposes such as `'Air Race show'` and `'Air Race/show'` were combined under `'ASHO'`. Additionally, all forms of public aircraft — including `'Public Aircraft - Federal'`, `'Public Aircraft - Local'`, and `'Public Aircraft - State'`, as well as codes like `'PUBS'` and `'PUBL'` — were standardized to `'Public Aircraft'`. This will help reduced fragmented data and provide more clarity in visualization**

###### 1.4.3.10: Column-->Broad.phase.of.flight

In [55]:
#checking proportion of null values
df['Broad.phase.of.flight'].isna().mean()*100

29.663095686301354

In [56]:
#checking unique values
df['Broad.phase.of.flight'].unique()

array(['Cruise', 'Unknown', 'Approach', 'Climb', 'Takeoff', 'Landing',
       'Taxi', 'Descent', 'Maneuvering', 'Standing', 'Go-around', 'Other',
       nan], dtype=object)

In [57]:
df['Broad.phase.of.flight'].value_counts()

Landing        15417
Takeoff        12476
Cruise         10255
Maneuvering     8134
Approach        6537
Climb           2028
Taxi            1958
Descent         1886
Go-around       1353
Standing         945
Unknown          543
Other            119
Name: Broad.phase.of.flight, dtype: int64

In [58]:
#Imputing using Unknown
df['Broad.phase.of.flight'].fillna('Unknown', inplace=True)

The `Broad.phase.of.flight` column had approximately **29.66%** missing values. Since the value `'Unknown'` already exists in the dataset, missing entries were imputed with `'Unknown'` to maintain consistency and minimize information loss. The column contains well-defined flight phases such as `'Landing'`, `'Takeoff'`, and `'Cruise'`, so no further standardization is necessary.


###### 1.4.3.11: Column-->Weather.Condition

In [59]:
#checking proportion of null values
df['Weather.Condition'].isna().mean()*100

3.974854821964381

In [60]:
#checking unique values
df['Weather.Condition'].unique()

array(['UNK', 'IMC', 'VMC', nan, 'Unk'], dtype=object)

In [61]:
df['Weather.Condition'].value_counts()

VMC    77149
IMC     5952
UNK      829
Unk      237
Name: Weather.Condition, dtype: int64

In [63]:
#Imputing and standardizing the Unk to UNK
df['Weather.Condition'] = df['Weather.Condition'].replace({'Unk': 'UNK'})
df['Weather.Condition'].fillna('UNK', inplace=True)

The `Weather.Condition` column has approximately **3.97%** missing values. Additionally, the column includes inconsistent representations of unknown values such as `'Unk'` and `'UNK'`.
To address this:
- All instances of `'Unk'` are standardized to `'UNK'`.
- Missing values are imputed with `'UNK'`, which already exists in the dataset.

## Methods

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87651 entries, 0 to 88888
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Investigation.Type      87651 non-null  object        
 1   Event.Date              87651 non-null  datetime64[ns]
 2   Location                87651 non-null  object        
 3   Country                 87651 non-null  object        
 4   Airport.Name            87651 non-null  object        
 5   Injury.Severity         87651 non-null  object        
 6   Aircraft.damage         87651 non-null  object        
 7   Aircraft.Category       87651 non-null  object        
 8   Make                    87651 non-null  object        
 9   Model                   87651 non-null  object        
 10  Amateur.Built           87651 non-null  object        
 11  Number.of.Engines       82433 non-null  float64       
 12  Engine.Type             87651 non-null  object

In [46]:
df.columns

Index(['Investigation.Type', 'Event.Date', 'Location', 'Country',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Make', 'Model', 'Amateur.Built',
       'Number.of.Engines', 'Engine.Type', 'Purpose.of.flight',
       'Total.Fatal.Injuries', 'Total.Serious.Injuries',
       'Total.Minor.Injuries', 'Total.Uninjured', 'Weather.Condition',
       'Broad.phase.of.flight'],
      dtype='object')

## Results

### Business Recommendation 1

### Business Recommendation 2

### Business Recommendation 3